In [ ]:
#load dependecies
from google.colab import drive
drive.mount('/content/drive/')

import tensorflow as tf
from tensorflow.keras import regularizers
import keras as k

import matplotlib.pyplot as plt
import torchsummary as summary
import numpy as np
np.random.seed(53702)
import pandas as pd
import os

!pip install shap --quiet
import shap
%matplotlib inline

!pip install scikit-survival --quiet

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, concatenate, Conv1D,Conv2D,AveragePooling2D,MaxPooling2D,LSTM
from keras.backend import dropout 
from keras.layers.merge import concatenate
from tensorflow.keras.utils import plot_model

!pip install lifelines --quiet
import lifelines

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
def loss_lik_efron(y_true,y_pred):
  #defining loss function
  time = y_true[:,0]
  event = y_true[:,1]

  y_pred= k.backend.flatten(y_pred)
  y_pred=tf.cast(y_pred,tf.float32)

  n = tf.shape(time)[0]
  sort_index=tf.nn.top_k(time,k=n,sorted=True).indices

  time = k.backend.gather(reference=time,indices = sort_index)
  event = k.backend.gather(reference=event,indices = sort_index)  
  y_pred = k.backend.gather(reference=y_pred,indices = sort_index)  

  time_event = time * event
  unique_ftime = tf.unique(tf.boolean_mask(tensor = time_event, mask = tf.greater(time_event, 0))).y
  m = tf.shape(unique_ftime)[0]
  tie_count=tf.unique_with_counts(tf.boolean_mask(time_event, tf.greater(time_event, 0))).count
  ind_matrix = k.backend.expand_dims(time,0) - k.backend.expand_dims(time,1)
  ind_matrix = k.backend.equal(x=ind_matrix,y=k.backend.zeros_like(ind_matrix))

  time_count = k.backend.cumsum(tf.unique_with_counts(time).count)
  time_count = k.backend.cast(time_count - k.backend.ones_like(time_count), dtype = tf.int32)
  ind_matrix = k.backend.gather(ind_matrix, time_count)
  ind_matrix=tf.cast(ind_matrix,'float32')


  event=tf.cast(event,'float32')
  tie_haz = k.backend.exp(y_pred) * event
  tie_haz = k.backend.dot(ind_matrix, k.backend.expand_dims(tie_haz))
  event_index = tf.math.not_equal(tie_haz,0)
  tie_haz = tf.boolean_mask(tie_haz, event_index)

  tie_risk = y_pred * event
  tie_risk = k.backend.dot(ind_matrix, k.backend.expand_dims(tie_risk))
  tie_risk = tf.boolean_mask(tie_risk, event_index)

  cum_haz = k.backend.dot(ind_matrix, k.backend.expand_dims(k.backend.exp(y_pred)))
  cum_haz = k.backend.reverse(tf.cumsum(k.backend.reverse(cum_haz, axes = 1)), axes = 1)
  cum_haz = tf.boolean_mask(cum_haz, event_index)

  global likelihood

  if likelihood is None:
    likelihood = tf.Variable(0., trainable = False) 

  j = tf.cast(0, dtype = tf.int32)
  def loop_cond(j,a,b,c,d,e):
    return j < m

  def loop_body(j, tc, tr, th, ch, lik):
    l = tc[j]
    l = k.backend.cast(l, dtype = tf.float32)
    J = tf.linspace(start = tf.cast(0, tf.float32), stop = l-1, num = tf.cast(l, tf.int32))/l 
    Dm = ch[j] - J*th[j]
    lik = lik + tr[j] - tf.math.reduce_sum(tf.math.log(Dm))
    one = k.backend.ones_like(j)
    j_new = j + one
    return(list([j_new, tc, tr, th, ch, lik]))

  loop_out = tf.while_loop(cond = loop_cond, body = loop_body,
                            loop_vars = list([j, tie_count, tie_risk, tie_haz, cum_haz, likelihood]))
  log_lik = loop_out[-1]

  return(tf.negative(log_lik))
likelihood = None

def load_data():
  #This function loads the data, filters out sections that have nans, or other 
  #missing values, and structures them so they can be used in the model afterwards
  os.chdir('/content/drive/MyDrive/Speciale/DL after 14-06/Combined')
  print(os.listdir())
  index_train=pd.read_csv("indices_train.csv",header=None)
  index_test=pd.read_csv("indices_test.csv",header=None)

  cens_lab = pd.read_csv("Comb_cens.txt")

  cens_lab = cens_lab[["days", "isdead"]]

  os.chdir('/content/drive/MyDrive/Speciale/DL after 14-06/Combined/')
    
  features_EOG_L=pd.read_csv('Combined_EOG_L.txt')
  features_EOG_R=pd.read_csv('Combined_EOG_R.txt')
  features_EEG=pd.read_csv('Combined_EEG.txt') 
  features_EMG=pd.read_csv('Combined_EMG.txt')
  features_Time=pd.read_csv('CombFeats.txt')

  imp = IterativeImputer(max_iter=10, random_state=0)
  IterativeImputer(random_state=0)

  os.chdir('/content/drive/MyDrive/Speciale/DL after 14-06/Combined_Reduced/')
  CombConf = pd.read_csv('CombConf' +".txt")

  imp.fit(CombConf)
  Imputed=imp.transform(CombConf)
  Comb_Conf_confounders=pd.DataFrame(Imputed,columns=CombConf.columns)

  Comb_t_feats = pd.read_csv('CombFeats' +".txt")

  Comb_Conf=pd.concat([CombConf['Age'],Comb_t_feats],1)

  Time_features_org= pd.read_csv('CombFeats' +".txt") 

  df=Comb_Conf_confounders
  df.Diabetes = df.Diabetes.round()
  df.Smoke_status = df.Smoke_status.round()
  df.Coffee = df.Coffee.round()
  df.Education = df.Education.round()
  df.Race = df.Race.round()

  cens_lab = pd.read_csv("Comb_cens.txt")

  combcens = pd.concat([cens_lab,df],1)

  TmpTimeFeat=Time_features_org.copy()

  df1=df.copy()

  features_Time=pd.read_csv('CombFeats.txt')

  features_Time=pd.concat([df1,features_Time],1)

  ###
  np.random.seed(53702)
  randnums= np.random.randint(0,10000,len(cens_lab))/10000
  features_EOG_L["RandomArray"]=randnums
  features_EOG_R["RandomArray2"]=randnums
  features_EEG["RandomArray3"]=randnums
  features_EMG["RandomArray5"]=randnums
  ### 
  print(np.shape(features_EOG_L))

  rows_with_nan = [index for index, row in features_EOG_L.iterrows() if row.isnull().any()]

  print(str(len(rows_with_nan)) + ' removed for having nans as features')

  features_EOG_L=features_EOG_L.drop(axis=0, index=rows_with_nan)
  features_EOG_R=features_EOG_R.drop(axis=0, index=rows_with_nan)
  features_EEG=features_EEG.drop(axis=0, index=rows_with_nan)
  features_EMG=features_EMG.drop(axis=0, index=rows_with_nan)
  features_Time=features_Time.drop(axis=0, index=rows_with_nan)

  cens_lab=cens_lab.drop(axis=0,index=rows_with_nan)

  Rem_more = [index for index, row in cens_lab.iterrows() if row.isnull().any()]

  print(str(len(Rem_more)) + ' removed for having empty censoring label')

  features_EOG_L=features_EOG_L.drop(axis=0, index=Rem_more)
  features_EOG_R=features_EOG_R.drop(axis=0, index=Rem_more)
  features_EEG=features_EEG.drop(axis=0, index=Rem_more)
  features_EMG=features_EMG.drop(axis=0, index=Rem_more)
  features_Time=features_Time.drop(axis=0, index=Rem_more)

  cens_lab=cens_lab.drop(axis=0,index=Rem_more)

  cens_lab=cens_lab.astype(int)

  np_EOG_L = features_EOG_L.to_numpy()
  np_EOG_R = features_EOG_R.to_numpy()
  np_EEG = features_EEG.to_numpy()
  np_EMG = features_EMG.to_numpy()
  np_Time=features_Time.to_numpy()

  np_cens=cens_lab.to_numpy()

  nptrainidx=index_train.to_numpy().astype(int);
  nptestidx=index_test.to_numpy().astype(int);

  x_train_EOG_L = np_EOG_L[nptrainidx,:]
  x_train_EOG_R = np_EOG_R[nptrainidx,:]
  x_train_EEG = np_EEG[nptrainidx,:]
  x_train_EMG = np_EMG[nptrainidx,:]
  x_train_Time= np_Time[nptrainidx,:]

  y_train_tmp=np_cens[nptrainidx,:]

  x_test_EOG_L = np_EOG_L[nptestidx,:]
  x_test_EOG_R = np_EOG_R[nptestidx,:]
  x_test_EEG = np_EEG[nptestidx,:]
  x_test_EMG = np_EMG[nptestidx,:]
  x_test_Time= np_Time[nptestidx,:]

  y_test_tmp=np_cens[nptestidx,:]

  x_train_EOG_L=x_train_EOG_L.reshape(np.shape(x_train_EOG_L)[0],np.shape(x_train_EOG_L)[2])
  x_train_EOG_R=x_train_EOG_R.reshape(np.shape(x_train_EOG_R)[0],np.shape(x_train_EOG_R)[2])
  x_train_EEG=x_train_EEG.reshape(np.shape(x_train_EEG)[0],np.shape(x_train_EEG)[2])
  x_train_EMG=x_train_EMG.reshape(np.shape(x_train_EMG)[0],np.shape(x_train_EMG)[2])
  x_train_Time=x_train_Time.reshape(np.shape(x_train_Time)[0],np.shape(x_train_Time)[2])


  y_train=y_train_tmp.reshape(np.shape(y_train_tmp)[0],np.shape(y_train_tmp)[2])

  x_test_EOG_L=x_test_EOG_L.reshape(np.shape(x_test_EOG_L)[0],np.shape(x_test_EOG_L)[2])
  x_test_EOG_R=x_test_EOG_R.reshape(np.shape(x_test_EOG_R)[0],np.shape(x_test_EOG_R)[2])
  x_test_EEG=x_test_EEG.reshape(np.shape(x_test_EEG)[0],np.shape(x_test_EEG)[2])
  x_test_EMG=x_test_EMG.reshape(np.shape(x_test_EMG)[0],np.shape(x_test_EMG)[2])
  x_test_Time=x_test_Time.reshape(np.shape(x_test_Time)[0],np.shape(x_test_Time)[2])

  y_test=y_test_tmp.reshape(np.shape(y_test_tmp)[0],np.shape(y_test_tmp)[2])

  comb_train=np.concatenate([x_train_EOG_L,x_train_EOG_R,x_train_EEG,x_train_EMG],axis=1)
  comb_test=np.concatenate([x_test_EOG_L,x_test_EOG_R,x_test_EEG,x_test_EMG],axis=1)

  feature_names=list(features_EOG_L.columns)+list(features_EOG_R.columns)+list(features_EEG.columns)+list(features_EMG.columns)
  print(np.shape(comb_train))
  shapes=[0,x_test_EOG_L.shape[1],x_test_EOG_R.shape[1],x_test_EEG.shape[1],x_test_EMG.shape[1]]

  return comb_train,comb_test,feature_names,shapes,y_train,y_test

def Init_Model(comb_train,shapes,num_L1,num_lr,dropOut):
    
    input1 = Input(shape=(np.shape(comb_train)[1],))

    def Splitter(tensor,shapes,iter):
        
        LowerLim=sum(shapes[:iter])
        UpperLim=sum(shapes[:iter+1])
        Seglength=int((UpperLim-LowerLim-1)/5)
        tensor=tensor[:,LowerLim:UpperLim]
        
        Out_Tensor_REM=tensor[:,0:Seglength]

        Out_Tensor_N3=tensor[:,Seglength:Seglength*2]
        Out_Tensor_N2=tensor[:,Seglength*2:Seglength*3]
        Out_Tensor_N1=tensor[:,Seglength*3:Seglength*4]
        Out_Tensor_Wake=tensor[:,Seglength*4:Seglength*5]


        Out_Tensor=tf.concat([Out_Tensor_REM, Out_Tensor_N3,Out_Tensor_N2,Out_Tensor_N1,Out_Tensor_Wake], 1)
        Dim1=int(Out_Tensor.get_shape()[1]/5)

        Out_Tensor=tf.reshape(Out_Tensor, (tf.shape(Out_Tensor_REM)[0],Dim1,5,1))
        print('shape')
        print(Out_Tensor.get_shape())
        return Out_Tensor

    def layerdesigner_Dense(input1,shapes,Num):
      #Dense layer structure
      input = tf.keras.layers.Lambda(lambda x:Splitter(x,shapes,Num))(input1)

      input_flattened = Flatten()(input)
      #First_Dense_1 = Dense(512,
      #                    activation='relu',
      #                    kernel_regularizer=regularizers.L1(num_L1))(input_flattened)
      #First_Dense_1 = Dropout(dropOut)(First_Dense_1)
      First_Dense = Dense(128,
                          activation='relu',
                          kernel_regularizer=regularizers.L1(num_L1))(input_flattened)
      First_Dense = Dropout(dropOut)(First_Dense)
      Second_Dense = Dense(64,
                          activation='relu',
                          kernel_regularizer=regularizers.L1(num_L1))(First_Dense)
      Second_Dense = Dropout(dropOut)(Second_Dense)  

      Second_Dense = Dense(32,
                    activation='relu',
                    kernel_regularizer=regularizers.L1(num_L1))(Second_Dense)
      Second_Dense = Dropout(dropOut)(Second_Dense) 

      Second_Dense = Dense(32,
                    activation='relu',
                    kernel_regularizer=regularizers.L1(num_L1))(Second_Dense)
      Second_Dense = Dropout(dropOut)(Second_Dense) 

      return Second_Dense      
    
    def layerdesigner_LSTM(input1,shapes,Num):
      #LSTM layer
      input = tf.keras.layers.Lambda(lambda x:Splitter(x,shapes,Num))(input1)

      Shape=input.get_shape()
      input_reshaped=tf.squeeze(input,[3])
      input_transposed=tf.transpose(input_reshaped,perm=[0,2,1])

      First_LSTM = LSTM(1024)(input_transposed)
      First_LSTM = Dropout(dropOut)(First_LSTM)

      #Second_LSTM = LSTM(32)(First_LSTM)
      #Second_LSTM = Dropout(dropOut)(Second_LSTM)  
      Dense_1 = Dense(64,
                    activation='relu',
                    kernel_regularizer=regularizers.L1(num_L1))(First_LSTM)
      Dense_1 = Dropout(dropOut)(Dense_1)

      Dense_2 = Dense(32,
                    activation='relu',
                    kernel_regularizer=regularizers.L1(num_L1))(Dense_1)
      Dense_2 = Dropout(dropOut)(Dense_2) 

      return Dense_2    

    def layerdesigner_CNN(input1,shapes,Num):
      #uncommented code here are other structures that have been tried
      input = tf.keras.layers.Lambda(lambda x:Splitter(x,shapes,Num))(input1)

      Shape=input.get_shape()
      print(Shape)

      First_CNN = Conv2D(32,(11,5),padding="same",activation='relu')(input)
      Max1=MaxPooling2D((1, 2))(First_CNN)

      second_CNN = Conv2D(16,(3,2),padding="same",activation='relu')(Max1)
      Max1=MaxPooling2D((1, 2))(second_CNN)

      # second_CNN = Conv2D(16,(3,2),padding="same",activation='relu')(Max1)
      # Max2=MaxPooling2D((2, 1))(second_CNN)

      Flatten_1 = Flatten()(Max1)

      #Dense_Out = Dense(128, 
      #                  activation = 'relu',
      #                  kernel_regularizer=regularizers.L1(num_L1))(Flatten_1)

      Dense_Out = Dense(32, 
                        activation = 'relu',
                        kernel_regularizer=regularizers.L1(num_L1))(Flatten_1)


          # EOG_L_Conv1 = Conv2D(32, (11,5), padding="same", activation='relu',input_shape=([101,5,1]))(EOG_L_input)
          # EOG_L_Conv1=MaxPooling2D(pool_size=(2,2))(EOG_L_Conv1)

          # EOG_L_Conv2 = Conv2D(64, (3,3), padding="same", activation='relu')(EOG_L_Conv1)
          # EOG_L_Conv2=MaxPooling2D(pool_size=(2,2))(EOG_L_Conv2)    

          # EOG_L_flattened = Flatten()(EOG_L_Conv2)

      return Dense_Out  
             
    if True:

      EOG_L=layerdesigner(input1,shapes,1)
      EOG_R=layerdesigner(input1,shapes,2)
      EEG=layerdesigner_Dense(input1,shapes,3)
      EMG=layerdesigner(input1,shapes,4)
    
    if False:
      EEG=layerdesigner_LSTM(input1,shapes,3)
    
    if True:
      EOG_L=layerdesigner_CNN(input1,shapes,1)
      EOG_R=layerdesigner_CNN(input1,shapes,2)
      EEG=layerdesigner_CNN(input1,shapes,3)
      EMG=layerdesigner_CNN(input1,shapes,4)
      Time=layerdesigner_Dense(input1,shapes,5)

    
    merge=concatenate([EEG,EOG_L,EOG_R,EEG,ECG,EMG])#,Dense_EOG_R_2,Dense_EEG_2])#,Dense_ECG_2,Dense_EMG_2])
    merge_dense = Dense(32,
                 activation='relu',
                 kernel_regularizer=regularizers.L1(num_L1))(merge)
    merge_dense = Dropout(dropOut)(merge_dense) 
    out = Dense(1)(merge)

    model = tf.keras.models.Model(inputs = input1, outputs=out)


    model.compile(loss=loss_lik_efron,
                  optimizer=tf.keras.optimizers.Adam(num_lr),
                    metrics=None)
    return model

def Run_Model(Min_CI,L1s,Lrs,batch_sizes,comb_train,comb_test,feature_names,shapes,model,y_train,y_test):
  #this function runs the model, and tests the model using the test set with the concordance index.
  #hyperparameters
  modelnumber=0
  print("Model:" + str(modelnumber) + "_BS:" + str(batch_sizes) + "_LR:" + str(Lrs) + "_L1:" + str(L1s))
  num_L1=L1s
  num_lr=Lrs
  num_epoch=200
  batch_size=batch_sizes
  
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

  history=model.fit(x=comb_train, y=y_train, 
              batch_size=batch_size, 
              epochs=num_epoch, 
              verbose=1, 
              validation_split=0.1,
              callbacks=[callback])
  
  predicts=model.predict(comb_test)
  df = pd.DataFrame(y_test, columns=['T', 'E'])
  df["predicts"]=predicts
  if bool(abs(pd.isnull(df["predicts"][1])-1)):
    try:
      cph = lifelines.fitters.coxph_fitter.CoxPHFitter().fit(df, 'T', 'E')
      cindex=round(lifelines.utils.concordance_index(df['T'], -cph.predict_partial_hazard(df), df['E'])*100,1)

      print(cindex)

      plt.plot(history.history['loss'])
      plt.plot(history.history['val_loss'])

      modelnumber += 1
      modelname="Model:" + str(modelnumber) + "_BS:" + str(batch_sizes) + "_LR:" + str(Lrs) + "_L1:" + str(L1s) + "_CI:" + str(cindex)
      TitleName="Model loss DNNSurv - BS:" + str(batch_sizes) + "_LR:" + str(Lrs) + "_L1:" + str(L1s) + "_CI:" + str(cindex)
      plt.title(TitleName)
      plt.ylabel('loss')
      plt.xlabel('epoch')
      plt.legend(['train', 'test'], loc='upper left')
      plt.show()

      df.to_csv('Ordering_predicts')
    except Exception as e:
      print(e) 
  return model

def main(Min_CI,comb_train,comb_test,feature_names,shapes,y_train,y_test):
  #hyperparameters
  L1s=0.001#[0.1,0.01,0.005,0.001,0.0005,0.0001]
  Lrs=0.005#[0.0005]#[1,0.1,0.01,0.005,0.001,0.0005,0.0001]
  batch_sizes=256#[32,64,128,256,512,1024]
  modelnumber=0
  dropOut=0.6

  for i in range(1):
    print('Iteration: ' + str(i+1))

    model=Init_Model(comb_train,shapes,L1s,Lrs,dropOut)
    plot_model(model, to_file='multiple_inputs.png',show_shapes=True)
    model=Run_Model(Min_CI,L1s,Lrs,batch_sizes,comb_train,comb_test,feature_names,shapes,model,y_train,y_test)
    
  return model
if __name__ == '__main__':
    if False:
      print('Loading Data...')
      comb_train,comb_test,feature_names,shapes,y_train,y_test=load_data()
      print('Loading Data Completed')
    Min_CI=62.5
    model=main(Min_CI,comb_train,comb_test,feature_names,shapes,y_train,y_test)